In [ ]:
import geopandas
import pandas as pd
import pgeocode
import plotly.graph_objects as go
nomi = pgeocode.Nominatim('ca')
from sklearn import cluster
import ipywidgets as widgets

In [ ]:
data = pd.read_csv('data/Connected member addresses for network map November 2022 - Sheet1.csv', header=None, 
                   names=['Member', 'Address', 'City', 'Postal Code'])
data.iloc[56]['Postal Code'] = 'T0L 0Z0' # Someone entered this wihout a space
data.drop([36, 38], axis=0, inplace=True) # Centers in Germany and France
data

In [ ]:
lats = []
lons = []
for i in data['Postal Code']:
    location = nomi.query_postal_code(i)
    lats.append(location['latitude'])
    lons.append(location['longitude'])
data['lat'] = lats
data['lon'] = lons
data['geometry'] = geopandas.points_from_xy(data['lat'], data['lon'])
data

In [ ]:
fig = go.Figure(data=go.Scattergeo(
    lat = data['lat'],
    lon = data['lon'],
    text = data['Member']
)
               )

fig.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig.show()
#fig.write_html('membermap.html')

In [ ]:
#!pip install kaleido
# fig.write_image('membermap.svg')

## Finding Scale distances on a 4x8 sheet of plywood

In [ ]:
'''
try:
    import haversine as hs
except:
    !pip install --user haversine
    import haversine as hs
'''
x_range = [data['lon'].max(), data['lon'].min()]
y_range = [data['lat'].max(), data['lat'].min()]
data['x'] = (data['lon'] - x_range[1]) / (x_range[0] - x_range[1])
data['y'] = (data['lat'] - y_range[1]) / (y_range[0] - y_range[1])
data['x inches'] = data['x'] * 46
data['y inches'] = data['y'] * 46
data

In [ ]:
import plotly.express as px
fig2 = px.scatter(data, x='x inches', y='y inches', hover_name='Member', hover_data=['Address', 'City', 'Postal Code'], height=800, width=400)
fig2.update_xaxes(range=[0, 48])
fig2.update_yaxes(range=[0, 96])
fig2.show()
fig.show()

## Clustering

What points should we consider clustering? With [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) we can similarly group nearby members.

An epsilon value of 1 equates to 111km of latitude and between 60 to 72km of longitude (given the latitude of the northernmest and southernmost members, respectively).

In [ ]:
import plotly
X = data[['lat', 'lon', 'Member']]

# Set value of epsilon here to determine radius of clustering algorithmn
epsilon = 0.4

cl = cluster.DBSCAN(eps=epsilon, min_samples=1).fit(X[['lat', 'lon']])
X['Label'] = cl.labels_
        
fig = go.Figure(data=go.Scattergeo(
    lat = data['lat'],
    lon = data['lon'],
    text = data['Member'],
    marker=dict(color=X['Label'], colorscale=plotly.colors.qualitative.Dark24))
)

fig.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig.show()
#fig.write_html('membermap.html')
display(X)
print(f"Unique clusters (for number of LEDs): {len(X['Label'].unique())}.\nEvery member that's within an ellipse of {round(111*epsilon,2)}km of latitude and ~{round(66*epsilon,2)}km of longitude with another member, belongs to the same cluster")

In [ ]:
# # Plotly discrete colour swatches
# import plotly
# plotly.colors.qualitative.swatches()